### Converting Universal Sentence Encoder Multilingual model to tensorflow-lite format

In [5]:
# !pip3 install --upgrade tensorflow==2.7.0 numpy==1.19.5 sentencepiece==0.1.95 scikit-learn==0.24.2 tensorflow_text==2.7.3 tqdm==4.61.0

In [1]:
import os
import requests
import tarfile
import tensorflow as tf
import tensorflow_text
import sentencepiece as spm
import numpy as np
from tqdm import tqdm

In [2]:
path_to_save_lite_model = os.path.join(os.getcwd(), "model.tflite")

model_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3?tf-hub-format=compressed"
model_name = model_url.split("?")[0]
model_name = model_name.split("/")[-2] + "_" + model_name.split("/")[-1] + ".tar.gz"
path_to_save = os.path.join(os.getcwd(), model_name)


In [3]:
def download(url, path_to_save):
    r = requests.get(url, stream=True)
    with open(path_to_save, 'wb') as f:
        for chunk in tqdm(r.iter_content(chunk_size=1024), desc=f"Downloading model from {url}"):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

def unarchive(path, path_to_extract):
    if path.endswith("tar.gz"):
        tar = tarfile.open(path, "r:gz")
        tar.extractall(path=path_to_extract)
        tar.close()

In [4]:
#downloading model
download(model_url, path_to_save)
saved_model_dir = path_to_save.replace("tar.gz", "")
#unarchiving model to use it
unarchive(path_to_save, saved_model_dir)

In [5]:
# converting model to tf-lite format
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()
open(path_to_save_lite_model, "wb").write(tflite_model)

339494052

In [6]:
def load_tflite_model(path_to_save_model):
    # Load the TFLite model and allocate tensors.
    interpreter = tf.lite.Interpreter(model_path=path_to_save_model)
    interpreter.allocate_tensors()

    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    return interpreter, input_details, output_details

def tf_lite_inference(input_str, interpreter, input_details, output_details):
    input_data = np.array([input_str])
    
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    
    output_data = interpreter.get_tensor(output_details[0]['index'])

    return output_data

In [7]:
interpreter, input_details, output_details = load_tflite_model(path_to_save_lite_model)

In [ ]:
input_str = "I'm just a text"

emb = tf_lite_inference(input_str, interpreter, input_details, output_details)
prinft(f"Embedding for the text {input_str} is {emb}")

### TODO
 - compare embeddings from tf-lite model with embeddings from regular model
 - Compare embeddings from different languages 